In [1]:
from itertools import combinations
import pandas as pd

In [3]:
dataset = pd.read_csv('../data/transactions.csv')

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52101 entries, 0 to 52100
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       52101 non-null  object
 1   1       46731 non-null  object
 2   2       43793 non-null  object
 3   3       41668 non-null  object
 4   4       39931 non-null  object
 5   5       38193 non-null  object
 6   6       36719 non-null  object
 7   7       35233 non-null  object
 8   8       33769 non-null  object
 9   9       32387 non-null  object
 10  10      31047 non-null  object
 11  11      29738 non-null  object
 12  12      28545 non-null  object
 13  13      27317 non-null  object
 14  14      26083 non-null  object
 15  15      24824 non-null  object
 16  16      23547 non-null  object
 17  17      22492 non-null  object
 18  18      21452 non-null  object
 19  19      20337 non-null  object
 20  20      19283 non-null  object
 21  21      18334 non-null  object
 22  22      17518 non-null

In [28]:
class Aprio:

    def __init__(self, dataset, min_support, min_confidence):
        self.min_support = min_support
        self.min_confidence = min_confidence
        self.transactions = self.list_from_dataframe(dataset)
        self.item_sets = self.generate_item_sets()
        self.frequent_items = self.get_frequent_itemsets()
        self.items_count = self.get_items_count()
        self.rules = self.generate_rules(self.frequent_items, self.min_confidence)
        
    def get_items_count(self):
        items_count = {}
        for transaction in self.transactions:
            for itemset in self.frequent_items:
                if itemset.issubset(transaction):
                    items_count[itemset] = items_count.get(itemset, 0) + 1
        return items_count

    """
        input: transactions in form of a dataframe
        output: a 2-D list of transactions [[item1, item2...], [item1, item3...]]
    """
    def list_from_dataframe(self, dataset):
        list_ = []
        for index in range(dataset.shape[0]):
            row = dataset.iloc[index].dropna()
            list_.append(list(row.values))
        return list_
    
    """
        needed data: 2-D list of transactions
        output: a list of all items in all transactions 
    """
    def generate_item_sets(self):
        return [frozenset([item]) for transaction in self.transactions for item in transaction]
    

    """
        removes items that does not verify the minimum support condition
    """
    def prune_min_supp(self, candidate_counts):
        return {itemset for itemset, count in candidate_counts.items() if count >= self.min_support}
        

    def prune_subsets(self,candidates, prev_frequent_itemsets):
        pruned_candidates = set()
        for candidate in candidates:
            is_valid = True
            subsets = combinations(candidate, len(candidate) - 1)
            for subset in subsets:
                if frozenset(subset) not in prev_frequent_itemsets:
                    is_valid = False
                    break
            if is_valid:
                pruned_candidates.add(candidate)
        return pruned_candidates


    def generate_next_candidates_set(self, prev_candidates, k):
        candidates = set()
        for itemset1 in prev_candidates:
            for itemset2 in prev_candidates:
                union_set = itemset1.union(itemset2)
                if len(union_set) == k:
                    candidates.add(union_set)
        return candidates


    """
        needed data: 2-D list of transactions
        output: list of union(F_k), frequent items in eatch iteration 
    """
    def get_frequent_itemsets(self):

        itemsets = self.item_sets.copy()
        frequent_itemsets = []
        
        k = 2

        while itemsets:

            candidate_counts = {}
            for transaction in self.transactions:
                for candidate in itemsets:
                    if candidate.issubset(transaction):
                        candidate_counts[candidate] = candidate_counts.get(candidate, 0) + 1
            
            frequent_itemsets_k = self.prune_min_supp(candidate_counts)
            
            candidates_k = self.generate_next_candidates_set(frequent_itemsets_k, k)
            
            candidates_k = self.prune_subsets(candidates_k, frequent_itemsets_k)
            
            frequent_itemsets.extend(frequent_itemsets_k)

            itemsets = candidates_k
            
            k += 1
        
        return frequent_itemsets

    def calculate_confidence(self, itemset, antecedent):
        return self.items_count[itemset] / self.items_count[antecedent]

    def calculate_lift(self, confidence, consequent):
        return confidence / self.calculate_support(consequent)

    def calculate_support(self, itemset):
        return self.items_count[itemset] / len(self.transactions)

    def generate_rules(self, frequent_itemsets, min_confidence):
        rules = []
        for itemset in frequent_itemsets:
            if len(itemset) > 1:
                itemset_list = list(itemset)
                for i in range(1, len(itemset)):
                    antecedent = frozenset(itemset_list[:i])
                    consequent = frozenset(itemset_list[i:])
                    confidence = self.calculate_confidence(itemset, antecedent)
                    lift = self.calculate_lift(confidence, consequent)
                    if confidence >= min_confidence:
                        rules.append({"antecedent": list(antecedent), "consequent": list(consequent), "confidence": confidence, "lift": lift})
        return rules

In [32]:
apr = Aprio(dataset.head(100), 5, 0.5)

In [33]:
apr.rules

[{'antecedent': ['22555'],
  'consequent': ['22556'],
  'confidence': 0.7142857142857143,
  'lift': 11.904761904761905},
 {'antecedent': ['22728'],
  'consequent': ['22727'],
  'confidence': 0.8571428571428571,
  'lift': 8.571428571428571},
 {'antecedent': ['22659'],
  'consequent': ['POST'],
  'confidence': 0.7142857142857143,
  'lift': 1.8315018315018314},
 {'antecedent': ['20679'],
  'consequent': ['15056BL'],
  'confidence': 1.0,
  'lift': 20.0},
 {'antecedent': ['84991'],
  'consequent': ['84992'],
  'confidence': 0.75,
  'lift': 10.714285714285714},
 {'antecedent': ['22728'],
  'consequent': ['22726'],
  'confidence': 0.8571428571428571,
  'lift': 12.244897959183671},
 {'antecedent': ['22375'],
  'consequent': ['84558A'],
  'confidence': 0.7142857142857143,
  'lift': 14.285714285714285},
 {'antecedent': ['22326'],
  'consequent': ['22630'],
  'confidence': 0.5,
  'lift': 3.846153846153846},
 {'antecedent': ['22698'],
  'consequent': ['22423'],
  'confidence': 1.0,
  'lift': 5.0},